In [6]:
import os
import geopandas as gpd
import folium
import shapely
import osmnx as ox
from osmnx import features
from shapely.geometry import LineString, MultiLineString

In [22]:
import os, io, zipfile, requests
import folium
import geopandas as gpd
import osmnx as ox
from shapely.geometry import LineString, MultiLineString
from xml.etree import ElementTree as ET

# -------- Settings --------
PLACE = "Donostia-San Sebastián, Gipuzkoa, Spain"
OUTPUT_HTML = "san_sebastian_mobility_map.html"
KMZ_URL = "https://www.gipuzkoa.eus/documents/33095840/33389605/IBILBIDE%2BSARE%2BOSOA.kmz/b16de1a6-a95f-90ea-7822-56f1a4a7a242?download=true&t=1655894335687"

# Faster network downloads
ox.settings.use_cache = True
ox.settings.log_console = False

# -------- Get area boundary (polygon) --------
area = ox.geocode_to_gdf(PLACE)
boundary = area.geometry.iloc[0]

# -------- Build networks --------
# 1) Walkable
G_walk = ox.graph_from_polygon(boundary, network_type="walk", simplify=True)
walk_edges = ox.graph_to_gdfs(G_walk, nodes=False)

# 2) Drive
G_drive = ox.graph_from_polygon(boundary, network_type="drive", simplify=True)
drive_edges = ox.graph_to_gdfs(G_drive, nodes=False)


# -------- Fetch & parse KMZ bike lanes --------
def fetch_kmz(url):
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return r.content


def parse_kmz_lines(kmz_bytes):
    with zipfile.ZipFile(io.BytesIO(kmz_bytes)) as zf:
        kml_name = next((n for n in zf.namelist() if n.lower().endswith(".kml")), None)
        if not kml_name:
            raise ValueError("No KML file found in KMZ")
        kml_bytes = zf.read(kml_name)

    root = ET.fromstring(kml_bytes)
    ns = {"kml": "http://www.opengis.net/kml/2.2"}
    lines = []
    for ls in root.findall(".//kml:LineString", ns):
        coords_el = ls.find("kml:coordinates", ns)
        if coords_el is None or not coords_el.text:
            continue
        pts = []
        for triplet in coords_el.text.strip().split():
            lon, lat, *_ = triplet.split(",")
            pts.append((float(lat), float(lon)))  # folium wants (lat, lon)
        if len(pts) >= 2:
            lines.append(LineString(pts))
    return gpd.GeoDataFrame(geometry=lines, crs="EPSG:4326")


kmz = fetch_kmz(KMZ_URL)
bike_edges = parse_kmz_lines(kmz)


# -------- Normalize columns --------
def to_lines_gdf(gdf, cols_keep=("highway", "name", "cycleway")):
    out = gdf.copy()
    for c in cols_keep:
        if c not in out.columns:
            out[c] = None
    return out[["geometry", *cols_keep]]


walk_edges = to_lines_gdf(walk_edges)
drive_edges = to_lines_gdf(drive_edges)
bike_edges = to_lines_gdf(bike_edges)  # no tags, just geometry

# -------- Make Folium map --------
centroid_latlon = (
    area.to_crs(4326).geometry.iloc[0].centroid.y,
    area.to_crs(4326).geometry.iloc[0].centroid.x,
)
m = folium.Map(location=centroid_latlon, zoom_start=13, tiles="cartodbpositron")

# Feature groups
fg_walk = folium.FeatureGroup(name="Walkable", show=False)
fg_drive = folium.FeatureGroup(name="Roads for cars", show=True)
fg_bike = folium.FeatureGroup(name="Bike lanes (KMZ)", show=True)
m.add_child(fg_drive)
m.add_child(fg_bike)
m.add_child(fg_walk)


# Add lines function
def add_lines(
    gdf, feature_group, weight=3, color="#222222", dash_array=None, tooltip=False
):
    if gdf.empty:
        return
    gdf_4326 = gdf.to_crs(4326)
    for _, row in gdf_4326.iterrows():
        geom = row.geometry

        def _coords_to_latlon(seq):
            return [(y, x) for x, y in seq]

        def _add_line(coords):
            folium.PolyLine(
                locations=_coords_to_latlon(coords),
                weight=weight,
                color=color,
                opacity=1.0,
                dash_array=dash_array,
            ).add_to(feature_group)

        if isinstance(geom, LineString):
            _add_line(list(geom.coords))
        elif isinstance(geom, MultiLineString):
            for part in geom.geoms:
                _add_line(list(part.coords))


# Draw networks
add_lines(drive_edges, fg_drive, weight=3, color="#2b2b2b")
add_lines(bike_edges, fg_bike, weight=3, color="#0a84ff")  # official KMZ bike lanes
add_lines(walk_edges, fg_walk, weight=2, color="#7e7e7e", dash_array="4,6")

# Boundary outline
folium.GeoJson(
    area.to_crs(4326),
    name="Municipal boundary",
    style_function=lambda x: {"fill": False, "color": "#777", "weight": 1},
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m.save(OUTPUT_HTML)
print(f"Saved: {os.path.abspath(OUTPUT_HTML)}")

Saved: d:\BikeSharing_Project\data\san_sebastian_mobility_map.html


In [ ]:
# print(area.head())
# print(area.columns)
# print(type(area))
# print(boundary)
print(type(walk_edges))
print(walk_edges.head())